In [0]:
from pyspark.sql import functions as F

In [0]:
%pip install s3fs

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
cn_raw = ( spark.read
    .option("header", "true")       
    .option("delimiter", ",")     
    .option("quote", '"')
    .option("escape",'"')                     
    .option("inferSchema", "true")
    .csv('s3://pprachan-eraneos-data/metadata/CN_modified.txt')
)

In [0]:
display(cn_raw.limit(10))

HS6,HS6_NAME,HS4,HS4_NAME,HS2,HS2_NAME
009000,"Total for countries whose data are confidential, broken down by origin and/or destination",0090,"Total for countries whose data are confidential, broken down by origin and/or destination",0,"Total for countries whose data are confidential, broken down by origin and/or destination"
010110,Pure-bred breeding horses and asses,0101,"Live horses, asses, mules and hinnies",1,LIVE ANIMALS
010111,Pure-bred breeding horses,0101,"Live horses, asses, mules and hinnies",1,LIVE ANIMALS
010119,Live horses (excl. pure-bred for breeding),0101,"Live horses, asses, mules and hinnies",1,LIVE ANIMALS
010120,"Live asses, mules and hinnies",0101,"Live horses, asses, mules and hinnies",1,LIVE ANIMALS
010121,Pure-bred breeding horses,0101,"Live horses, asses, mules and hinnies",1,LIVE ANIMALS
010129,Live horses (excl. pure-bred for breeding),0101,"Live horses, asses, mules and hinnies",1,LIVE ANIMALS
010130,Live asses,0101,"Live horses, asses, mules and hinnies",1,LIVE ANIMALS
010190,Live mules and hinnies,0101,"Live horses, asses, mules and hinnies",1,LIVE ANIMALS
010210,Pure-bred breeding bovines,0102,Live bovine animals,1,LIVE ANIMALS


# Find the HS codes related to citrus 

In [0]:
display(
        cn_raw.filter(F.lower(F.col("HS2_NAME")).contains("citrus"))
        .select("HS2","HS2_NAME")
        .distinct()
)

HS2,HS2_NAME
8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS


In [0]:
hs2_list_df = (
    cn_raw
        .filter(F.lower(F.col("HS2_NAME")).contains("citrus"))
        .select("HS2")
        .distinct()
)
hs2_list = [row["HS2"] for row in hs2_list_df.collect()]

HS2 codes: 
* 08: EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS

In [0]:
display(
    cn_raw
        .filter(
            (F.col("HS2").isin(hs2_list)) &
            (F.lower(F.col("HS4_NAME")).contains("citrus"))
        )
        .select("HS4", "HS4_NAME")
        .distinct()
        .orderBy(F.col("HS4"))
)


HS4,HS4_NAME
0805,"Citrus fruit, fresh or dried"
0810,"Fresh strawberries, raspberries, blackberries, back, white or red currants, gooseberries and other edible fruits (excl. nuts, bananas, dates, figs, pineapples, avocados, guavas, mangoes, mangosteens, papaws ""papayas"", citrus fruit, grapes, melons, apples, pears, quinces, apricots, cherries, peaches, plums and sloes)"
0813,"Dried apricots, prunes, apples, peaches, pears, papaws ""papayas"", tamarinds and other edible fruits, and mixtures of edible and dried fruits or of edible nuts (excl. nuts, bananas, dates, figs, pineapples, avocados, guavas, mangoes, mangosteens, citrus fruit and grapes, unmixed)"
0814,"Peel of citrus fruit or melons, incl. watermelons, fresh, frozen, dried or provisionally preserved in brine, or in water with other additives"


In [0]:
# Keep only 0805 and 0814
hs4_list = ['0805','0814']

In [0]:
display(
    cn_raw
        .filter(
            (F.col('HS4').isin(hs4_list)) 
            # & 
            # (F.col('HS6_NAME').contains('citrus'))
        )
        .orderBy('HS6')
)

HS6,HS6_NAME,HS4,HS4_NAME,HS2,HS2_NAME
080510,Fresh or dried oranges,0805,"Citrus fruit, fresh or dried",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS
080520,"Fresh or dried mandarins incl. tangerines and satsumas, clementines, wilkings and similar citrus hybrids",0805,"Citrus fruit, fresh or dried",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS
080521,Fresh or dried mandarins incl. tangerines and satsumas (excl. clementines),0805,"Citrus fruit, fresh or dried",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS
080522,Fresh or dried clementines incl. monreales,0805,"Citrus fruit, fresh or dried",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS
080529,Fresh or dried wilkings and similar citrus hybrids,0805,"Citrus fruit, fresh or dried",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS
080530,"Fresh or dried lemons ""Citrus limon, Citrus limonum"" and limes ""Citrus aurantifolia""",0805,"Citrus fruit, fresh or dried",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS
080540,Fresh or dried grapefruit,0805,"Citrus fruit, fresh or dried",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS
080550,"Fresh or dried lemons ""Citrus limon, Citrus limonum"" and limes ""Citrus aurantifolia, Citrus latifolia""",0805,"Citrus fruit, fresh or dried",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS
080590,"Fresh or dried citrus fruit (excl. oranges, lemons ""Citrus limon, Citrus limonum"", limes ""Citrus aurantifolia, Citrus latifolia"", grapefruit, mandarins, incl. tangerines and satsumas, clementines, wilkings and similar citrus hybrids)",0805,"Citrus fruit, fresh or dried",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS
081400,"Peel of citrus fruit or melons, incl. watermelons, fresh, frozen, dried or provisionally preserved in brine, or in water with other additives",0814,"Peel of citrus fruit or melons, incl. watermelons, fresh, frozen, dried or provisionally preserved in brine, or in water with other additives",8,EDIBLE FRUIT AND NUTS; PEEL OF CITRUS FRUIT OR MELONS


In [0]:
hs6_list_df = (
    cn_raw
        .filter(
            (F.col('HS4').isin(hs4_list))
        )
        .select('HS6', 'HS6_NAME', 'HS4', 'HS4_NAME')
        .distinct()
)

In [0]:
display(hs6_list_df)

HS6,HS6_NAME,HS4,HS4_NAME
080522,Fresh or dried clementines incl. monreales,0805,"Citrus fruit, fresh or dried"
080529,Fresh or dried wilkings and similar citrus hybrids,0805,"Citrus fruit, fresh or dried"
080521,Fresh or dried mandarins incl. tangerines and satsumas (excl. clementines),0805,"Citrus fruit, fresh or dried"
080520,"Fresh or dried mandarins incl. tangerines and satsumas, clementines, wilkings and similar citrus hybrids",0805,"Citrus fruit, fresh or dried"
080530,"Fresh or dried lemons ""Citrus limon, Citrus limonum"" and limes ""Citrus aurantifolia""",0805,"Citrus fruit, fresh or dried"
080540,Fresh or dried grapefruit,0805,"Citrus fruit, fresh or dried"
080550,"Fresh or dried lemons ""Citrus limon, Citrus limonum"" and limes ""Citrus aurantifolia, Citrus latifolia""",0805,"Citrus fruit, fresh or dried"
081400,"Peel of citrus fruit or melons, incl. watermelons, fresh, frozen, dried or provisionally preserved in brine, or in water with other additives",0814,"Peel of citrus fruit or melons, incl. watermelons, fresh, frozen, dried or provisionally preserved in brine, or in water with other additives"
080510,Fresh or dried oranges,0805,"Citrus fruit, fresh or dried"
080590,"Fresh or dried citrus fruit (excl. oranges, lemons ""Citrus limon, Citrus limonum"", limes ""Citrus aurantifolia, Citrus latifolia"", grapefruit, mandarins, incl. tangerines and satsumas, clementines, wilkings and similar citrus hybrids)",0805,"Citrus fruit, fresh or dried"


In [0]:
(hs6_list_df
    .coalesce(1)
    .write.mode('overwrite')
    .option('header', 'true')
    .option('delimiter', ';')
    .csv('s3://pprachan-eraneos-data/hs6_citrus/')
)